In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, callbacks, optimizers, models
from keras.models import Sequential

Using TensorFlow backend.


In [5]:
num_of_classes = 46
classifier = Sequential()
classifier.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,1)))
classifier.add(layers.MaxPooling2D((2,2)))
classifier.add(layers.Conv2D(64, (3,3), activation='relu'))
classifier.add(layers.MaxPooling2D((2,2)))
classifier.add(layers.Conv2D(64,(3,3), activation='relu'))
classifier.add(layers.Flatten())
classifier.add(layers.Dense(num_of_classes, activation='relu'))
classifier.add(layers.Dense(num_of_classes, activation='softmax'))
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# opt = optimizers.Adam(lr=0.0015, beta_1=0.9, beta_2=0.99, decay=0.0, amsgrad=False)
# classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
training_set = train_datagen.flow_from_directory('Train',
                                                   target_size=(32, 32),
                                                    batch_size=32,
                                                    color_mode = 'grayscale',
                                                    class_mode='categorical')

Found 78200 images belonging to 46 classes.


In [5]:
test_set = test_datagen.flow_from_directory('Test',
                                                target_size=(32, 32),
                                                
                                                color_mode = 'grayscale',
                                                batch_size=1,
                                                shuffle = False,
                                                seed = 2076,
                                                class_mode='categorical')

Found 13800 images belonging to 46 classes.


In [6]:
sample_size = training_set.n
batch_size = 32
history =classifier.fit_generator(training_set,
                            steps_per_epoch=sample_size // batch_size,
                            epochs=1,
                            validation_data=test_set,
                            validation_steps=13800,
                            
                            )


Epoch 1/1
2443/2443 [==============================] - 104s 43ms/step - loss: 1.0891 - accuracy: 0.6874 - val_loss: 1.6366e-04 - val_accuracy: 0.8563


In [7]:
classifier.save('anvil.h5')

In [8]:
from keras.models import load_model
from keras_preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf


# load model
model_path = 'new.h5'
convnet = load_model(model_path)
graph = tf.get_default_graph()

nepali=['ka','khaa','ga','4','5','6','7','8','9','10','क','12','13','14','15','16','17','18','19','20','21','kha','23','24','25','26','27','28','29','30','31','32','33','34','35','36','o', '१' ,'२' , '३', '४', '५', '६', '७', '८', '९']

def predict_character(image_file):
    global graph
    with graph.as_default():
        image_loaded = load_img(image_file,target_size=(32,32),color_mode='grayscale')
        img_arr = (img_to_array(image_loaded)/255.0).reshape(1,32,32,1)
        probabilities = convnet.predict(img_arr)
        pred = np.argmax(probabilities)
        return nepali[pred], np.amax(probabilities)

In [9]:
pip install anvil-uplink

  Created wheel for ws4py: filename=ws4py-0.3.4-cp37-none-any.whl size=41816 sha256=7c1766dec8f8d1f4a78af7d8cd578d3c22f528794abef98480c9656097606c2d
  Stored in directory: C:\Users\Krishna\AppData\Local\pip\Cache\wheels\19\1f\0d\beff5822af761b66067b5e0b251a9c66af3ae15828ee9a8f15
Successfully built ws4py
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
import anvil.server
anvil.server.connect("SRBPI7YIPNBWMTEJTEQT7SYQ-VC4C4VHCHESN7SXI")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [6]:
import PIL
import anvil.media
@anvil.server.callable

def classify_image(file):
    nepali=['ka','khaa','ga','4','5','6','7','8','9','10','क','12','13','14','15','16','17','18','19','20','21','kha','23','24','25','26','27','28','29','30','31','32','33','34','35','36','o', '१' ,'२' , '३', '४', '५', '६', '७', '८', '९']
    with anvil.media.TempFile(file) as filename:
        img = load_img(filename)
    img = img.resize((1,32,32,1), resample = PIL.Image.BICUBIC)
    img_arr = img_to_array(img)
    probabilities = convnet.predict(img_arr)
    pred = np.argmax(probabilities)
    return nepali[pred], np.amax(probabilities)